
# connect to google drive



In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# import packages

In [47]:
import os

import pandas as pd

In [48]:
# os.listdir('/content/drive/Shareddrives/數位醫學/case 1')
os.chdir('/content/drive/Shareddrives/數位醫學/case 1')
# print(os.getcwd())
# os.listdir()

# sample data

In [49]:
data_path = 'Case Presentation 1 Data'
data = {dataset:{name:open(os.path.join(data_path,dataset,name)).read() for name in os.listdir(os.path.join(data_path,dataset))} for dataset in os.listdir(data_path)}

In [50]:
print(data.keys())

dict_keys(['Test_Intuitive', 'Validation', 'Train_Textual'])


In [51]:
# data['Train_Textual'].keys()

In [52]:
data['Train_Textual']['Y_ID_147.txt']

"\n546299534 | CRH | 54433044 | | 6629491 | 2/5/2003 12:00:00 AM | supraspinatus tendonitis | | DIS | Admission Date: 6/11/2003 Report Status:\n\nDischarge Date: 4/17/2003\n****** DISCHARGE ORDERS ******\nDIVERS , EMORY O 084-86-53-0\nLisson Co Wa\nService: MED\nDISCHARGE PATIENT ON: 3/25/03 AT 12:00 M\nCONTINGENT UPON Not Applicable\nWILL D/C ORDER BE USED AS THE D/C SUMMARY: YES\nAttending: MARTELL , MAYNARD CALEB , M.D.\nCODE STATUS:\nFull code\nDISPOSITION: Home w/ services\nDISCHARGE MEDICATIONS:\nECASA ( ASPIRIN ENTERIC COATED ) 81 MG PO QD\nAlert overridden: Override added on 3/25/03 by\nCURENTON , FAUSTO JOSH , M.D.\nPOTENTIALLY SERIOUS INTERACTION: WARFARIN & ASPIRIN\nReason for override: will follow\nLASIX ( FUROSEMIDE ) 80 MG PO BID\nLOPRESSOR ( METOPROLOL TARTRATE ) 100 MG PO TID\nFood/Drug Interaction Instruction\nTake consistently with meals or on empty stomach.\nCOUMADIN ( WARFARIN SODIUM ) 4 MG PO QD\nFood/Drug Interaction Instruction\nNo high Vitamin-K containing foods

## text preprocssing

In [53]:
import re
for key, value in data['Train_Textual'].items():
  value = re.sub(r'[\n|*:,.()]' ,' ',value.lower())
  value = value.split(' ')
  data['Train_Textual'][key] = value

for key, value in data['Test_Intuitive'].items():
  value = re.sub(r'[\n|*:,.()]' ,' ',value.lower())
  value = value.split(' ')
  data['Test_Intuitive'][key] = value

for key, value in data['Validation'].items():
  value = re.sub(r'[\n|*:,.()]' ,' ',value.lower())
  value = value.split(' ')
  data['Validation'][key] = value

In [54]:
# data['Train_Textual']['Y_ID_97.txt']

# keyword

In [55]:
keyword_path = 'keyword'
os.listdir(keyword_path)

['Term_147_0.xlsx',
 'Term_150_0.xlsx',
 'Term_Recognizing Obesity and Comorbidities in Sparse Data.gsheet',
 'Term_Refers.xls']

In [56]:
df_keyword = pd.DataFrame()
for i in os.listdir(keyword_path):
  if 'xlsx' in i:
    df_keyword = df_keyword.append(pd.read_excel(os.path.join(keyword_path,i))[["英文名稱","中文名稱"]])
  else:
    print(i)

set_keyword = set(df_keyword['英文名稱'].apply(lambda x:x.lower()))

Term_Recognizing Obesity and Comorbidities in Sparse Data.gsheet
Term_Refers.xls


In [57]:
temp = set()
for item in set_keyword:
  if ';' in item:
    print(item) if '17' in item else 0
    temp = temp.union({y for x in item.split(';') for y in x.split(' ')})

17;20-desmolase deficiency
brownism (brown1735-88); brunonianism; brunonian theory


In [58]:
set_keyword = set_keyword.union(temp)

In [59]:
# set_keyword.remove(False)

In [60]:
# set_keyword

# obese

## Mention obese or not
Using crosstable check

In [61]:
obesity = {'obese', 'obesity', 'overweight', 
          #  'fat'
}

In [62]:
df_obese_mention = pd.DataFrame()
for key,value in data['Train_Textual'].items():
  df_obese_mention.loc[key,'file_type'] = True if key[0] == 'Y' else False
  df_obese_mention.loc[key,'obesity mentioned'] = sum([i.lower() in value for i in obesity]) > 0
  for keyword in obesity:
    df_obese_mention.loc[key, keyword] = value.count(keyword)
df_obese_mention.head()

,file_type,obesity mentioned,obese,obesity,overweight
U_ID_71.txt,False,False,0.0,0.0,0.0
Y_ID_134.txt,True,True,1.0,1.0,0.0
U_ID_69.txt,False,False,0.0,0.0,0.0
U_ID_59.txt,False,False,0.0,0.0,0.0
U_ID_151.txt,False,False,0.0,0.0,0.0


In [63]:
pd.crosstab(df_obese_mention['file_type'], df_obese_mention['obesity mentioned'], rownames=['file mentioned'], colnames=['obesity mentioned'])

obesity mentioned,False,True
file mentioned,,
False,185,15
True,58,142


In [64]:
df_test = pd.DataFrame()
for key,value in data['Test_Intuitive'].items():
  df_test.loc[key,'file_type'] = True if key[0] == 'Y' else False
  df_test.loc[key,'obesity mentioned'] = sum([i.lower() in value for i in obesity]) > 0
  for keyword in obesity:
    df_test.loc[key, keyword] = value.count(keyword)

df_test.head()

,file_type,obesity mentioned,obese,obesity,overweight
N_ID_127.txt,False,False,0.0,0.0,0.0
Y_ID_84.txt,True,True,1.0,3.0,0.0
Y_ID_133.txt,True,True,2.0,2.0,0.0
Y_ID_132.txt,True,True,1.0,3.0,0.0
Y_ID_128.txt,True,True,1.0,0.0,0.0


In [65]:
pd.crosstab(df_test['file_type'], df_test['obesity mentioned'], rownames=['obesity'], colnames=['obesity mentioned'])

obesity mentioned,False,True
obesity,,
False,200,0
True,9,191


In [66]:
df_vaild = pd.DataFrame()
for key,value in data['Validation'].items():
  df_vaild.loc[key,'obesity mentioned'] = sum([i.lower() in value for i in obesity]) > 0
  for keyword in obesity:
    df_vaild.loc[key, keyword] = value.count(keyword)
df_vaild.head()

,obesity mentioned,obese,obesity,overweight
ID_1201.txt,False,0.0,0.0,0.0
ID_1213.txt,False,0.0,0.0,0.0
ID_1198.txt,False,0.0,0.0,0.0
ID_1229.txt,False,0.0,0.0,0.0
ID_1162.txt,True,2.0,2.0,0.0


## there is keywords, such as 'obese' , in these files.

In [67]:
dict_Y = {}
dict_U = {}
for key,value in data['Train_Textual'].items():
  if 'Y' in key:
    dict_Y[key] = value
  else:
    dict_U[key] = value

In [68]:
for key, value in dict_U.items():
  if sum([i.lower() in value for i in obesity]) != 0:
    print(key, sum([i.lower() in value for i in obesity]))

U_ID_716.txt 2
U_ID_851.txt 1
U_ID_882.txt 1
U_ID_725.txt 1
U_ID_891.txt 1
U_ID_747.txt 2
U_ID_874.txt 1
U_ID_713.txt 1
U_ID_737.txt 2
U_ID_855.txt 2
U_ID_861.txt 1
U_ID_728.txt 1
U_ID_877.txt 1
U_ID_884.txt 1
U_ID_740.txt 2


In [69]:
# [key if (sum([i.lower() in value for i in obesity]) == 0) for key, value in dict_Y.items()]
for key, value in dict_Y.items():
  if sum([i.lower() in value for i in obesity]) == 0:
    print(key, sum([i.lower() in value for i in obesity]))

Y_ID_21.txt 0
Y_ID_6.txt 0
Y_ID_20.txt 0
Y_ID_1054.txt 0
Y_ID_996.txt 0
Y_ID_873.txt 0
Y_ID_883.txt 0
Y_ID_746.txt 0
Y_ID_960.txt 0
Y_ID_1076.txt 0
Y_ID_1027.txt 0
Y_ID_932.txt 0
Y_ID_1012.txt 0
Y_ID_722.txt 0
Y_ID_1079.txt 0
Y_ID_976.txt 0
Y_ID_941.txt 0
Y_ID_1043.txt 0
Y_ID_1072.txt 0
Y_ID_1067.txt 0
Y_ID_943.txt 0
Y_ID_1031.txt 0
Y_ID_935.txt 0
Y_ID_890.txt 0
Y_ID_1063.txt 0
Y_ID_991.txt 0
Y_ID_1050.txt 0
Y_ID_892.txt 0
Y_ID_715.txt 0
Y_ID_854.txt 0
Y_ID_857.txt 0
Y_ID_1037.txt 0
Y_ID_945.txt 0
Y_ID_897.txt 0
Y_ID_954.txt 0
Y_ID_876.txt 0
Y_ID_734.txt 0
Y_ID_929.txt 0
Y_ID_750.txt 0
Y_ID_1065.txt 0
Y_ID_1089.txt 0
Y_ID_868.txt 0
Y_ID_949.txt 0
Y_ID_987.txt 0
Y_ID_983.txt 0
Y_ID_915.txt 0
Y_ID_1047.txt 0
Y_ID_936.txt 0
Y_ID_739.txt 0
Y_ID_880.txt 0
Y_ID_1006.txt 0
Y_ID_726.txt 0
Y_ID_1002.txt 0
Y_ID_909.txt 0
Y_ID_978.txt 0
Y_ID_904.txt 0
Y_ID_921.txt 0
Y_ID_968.txt 0


## find other keywords
other_key was mentioned in paper, *Recognizing Obesity and Comorbidities in Sparse Data*,  table 1.


In [70]:
other_key = {'Asthma','Atherosclerotic CV disease', 'CAD', 'Congestive heart failure', 'CHF','Depression',
      'Diabetes mellitus','Gallstones','cholecystectomy','GERD','Gout','Hypercholesterolemia',
      'Hypertension','HTN','Hypertriglyceridemia','Obstructive sleep apnea','OSA','Osteoarthritis','OA',
      'Peripheral vascular disease','PVD','Venous insufficiency',
      # 'fatty liver','edema','hypertension','cholesterol','diabetes mellitus','DM','Cardiology','hypercholesterolemia','lipidemia','hypothyroidism'
      }
      

## analysis

In [71]:
assemble_list_Y = []
for value in dict_Y.values():
  assemble_list_Y += value


assemble_list_U = []
for value in dict_U.values():
  assemble_list_U += value


In [72]:
dict_out1 = {}
for value in dict_Y.values():
  for key in set_keyword.intersection(set(assemble_list_Y)):
    if key in value:
      dict_out1[key] = dict_out1.get(key, 0) + 1

for value in dict_U.values():
  for key in set_keyword.intersection(set(assemble_list_U)):
    if key in value:
      dict_out1[key] = dict_out1.get(key, 0) + 1

In [73]:
dict_out2 = {}
for key, value in dict_out1.items():
  if (value < 250) and (value > 150):
    dict_out2[key] = value

In [74]:
other_key = other_key.union(set_keyword.intersection(set(dict_out2.keys())))

In [75]:
for key,value in data['Train_Textual'].items():
  for keyword in other_key:
    df_obese_mention.loc[key,keyword] = value.count(keyword)
    # df_obese_mention.loc[key, keyword] = 1 if keyword in value else 0
df_obese_mention.head()

,file_type,obesity mentioned,obese,obesity,overweight,Hypercholesterolemia,pressure,physical,Peripheral vascular disease,insulin,Hypertension,end,CHF,secondary,hypertension,room,failure,normal,mild,allergy,illness,abdomen,OA,cholecystectomy,full,Gout,Venous insufficiency,order,without,soft,acute,OSA,PVD,potassium,Obstructive sleep apnea,Congestive heart failure,HTN,out,Asthma,continued,Gallstones,Atherosclerotic CV disease,GERD,chronic,Osteoarthritis,i,rhythm,Hypertriglyceridemia,CAD,study,artery,renal,edema,b,Depression,Diabetes mellitus
U_ID_71.txt,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,1.0,0.0,0.0
Y_ID_134.txt,True,True,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0
U_ID_69.txt,False,False,0.0,0.0,0.0,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0,4.0,1.0,0.0,0.0
U_ID_59.txt,False,False,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
U_ID_151.txt,False,False,0.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,3.0,4.0,1.0,4.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,4.0,4.0,4.0,1.0,0.0,0.0


In [76]:
for key,value in data['Test_Intuitive'].items():
  for keyword in other_key:
    df_test.loc[key,keyword] = value.count(keyword)
    # df_test.loc[key, keyword] = 1 if keyword in value else 0
df_test.head()

,file_type,obesity mentioned,obese,obesity,overweight,Hypercholesterolemia,pressure,physical,Peripheral vascular disease,insulin,Hypertension,end,CHF,secondary,hypertension,room,failure,normal,mild,allergy,illness,abdomen,OA,cholecystectomy,full,Gout,Venous insufficiency,order,without,soft,acute,OSA,PVD,potassium,Obstructive sleep apnea,Congestive heart failure,HTN,out,Asthma,continued,Gallstones,Atherosclerotic CV disease,GERD,chronic,Osteoarthritis,i,rhythm,Hypertriglyceridemia,CAD,study,artery,renal,edema,b,Depression,Diabetes mellitus
N_ID_127.txt,False,False,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,3.0,0.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,8.0,2.0,0.0,0.0,5.0,1.0,0.0,2.0,5.0,0.0,0.0
Y_ID_84.txt,True,True,1.0,3.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,4.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,5.0,1.0,1.0,0.0,0.0
Y_ID_133.txt,True,True,2.0,2.0,0.0,0.0,1.0,2.0,0.0,4.0,0.0,0.0,0.0,1.0,2.0,2.0,1.0,2.0,1.0,0.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0,0.0,7.0,1.0,0.0,0.0,1.0,3.0,0.0,1.0,3.0,0.0,0.0
Y_ID_132.txt,True,True,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
Y_ID_128.txt,True,True,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0


In [77]:
for key,value in data['Validation'].items():
  for keyword in other_key:
    df_vaild.loc[key,keyword] = value.count(keyword)
    # df_vaild.loc[key, keyword] = 1 if keyword in value else 0
df_vaild.head()

,obesity mentioned,obese,obesity,overweight,Hypercholesterolemia,pressure,physical,Peripheral vascular disease,insulin,Hypertension,end,CHF,secondary,hypertension,room,failure,normal,mild,allergy,illness,abdomen,OA,cholecystectomy,full,Gout,Venous insufficiency,order,without,soft,acute,OSA,PVD,potassium,Obstructive sleep apnea,Congestive heart failure,HTN,out,Asthma,continued,Gallstones,Atherosclerotic CV disease,GERD,chronic,Osteoarthritis,i,rhythm,Hypertriglyceridemia,CAD,study,artery,renal,edema,b,Depression,Diabetes mellitus
ID_1201.txt,False,0.0,0.0,0.0,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,8.0,6.0,1.0,1.0,0.0,0.0
ID_1213.txt,False,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8.0,0.0,1.0,0.0,0.0,0.0
ID_1198.txt,False,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,4.0,0.0,0.0
ID_1229.txt,False,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,3.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0
ID_1162.txt,True,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


# model 

In [78]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100000,max_depth=10)
model.fit(df_obese_mention.iloc[:,2:],df_obese_mention.iloc[:,0].apply(lambda x : 'True' if x else 'False'))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [79]:
model.score(df_obese_mention.iloc[:,2:],df_obese_mention.iloc[:,0].apply(lambda x : 'True' if x else 'False'))

0.965

In [80]:
df_predict = df_obese_mention[['file_type','obesity mentioned']]
df_predict.loc[:,'predict'] = model.predict(df_obese_mention.iloc[:,2:])
df_predict
pd.crosstab(df_predict['file_type'], df_predict['predict'], rownames=['obesity'], colnames=['predict'])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


predict,False,True
obesity,,
False,200,0
True,14,186


In [81]:
model.score(df_test.iloc[:,2:],df_test.iloc[:,0].apply(lambda x : 'True' if x else 'False'))

0.925

In [82]:
df_predict = df_test.copy()
df_predict.loc[:,'predict'] = model.predict(df_test.iloc[:,2:])
df_predict
pd.crosstab(df_predict['file_type'], df_predict['predict'], rownames=['obesity'], colnames=['predict'])

predict,False,True
obesity,,
False,185,15
True,15,185


In [83]:
df_ouput = df_vaild.copy()
df_ouput['Obesity'] = model.predict(df_vaild.iloc[:,1:])
df_ouput['Obesity'] = df_ouput['Obesity'].apply(lambda x : 1 if x == 'True' else 0)


In [84]:
df_ouput.index.name = 'Filename'

In [85]:
df_ouput['Obesity'].head()

Filename
ID_1201.txt    0
ID_1213.txt    0
ID_1198.txt    0
ID_1229.txt    0
ID_1162.txt    1
Name: Obesity, dtype: int64

In [86]:
df_ouput['Obesity'].to_csv('ouput_version2.csv')

In [87]:
df_ouput['Obesity'].sum()

19

In [88]:
df_obese_mention.columns[2:]

Index(['obese', 'obesity', 'overweight', 'Hypercholesterolemia', 'pressure',
       'physical', 'Peripheral vascular disease', 'insulin', 'Hypertension',
       'end', 'CHF', 'secondary', 'hypertension', 'room', 'failure', 'normal',
       'mild', 'allergy', 'illness', 'abdomen', 'OA', 'cholecystectomy',
       'full', 'Gout', 'Venous insufficiency', 'order', 'without', 'soft',
       'acute', 'OSA', 'PVD', 'potassium', 'Obstructive sleep apnea',
       'Congestive heart failure', 'HTN', 'out', 'Asthma', 'continued',
       'Gallstones', 'Atherosclerotic CV disease', 'GERD', 'chronic',
       'Osteoarthritis', 'i', 'rhythm', 'Hypertriglyceridemia', 'CAD', 'study',
       'artery', 'renal', 'edema', 'b', 'Depression', 'Diabetes mellitus'],
      dtype='object')

In [89]:
df_feature = pd.DataFrame()
df_feature['keyword'] = df_obese_mention.columns[2:]
df_feature['feature importance'] = model.feature_importances_

In [90]:
df_feature.sort_values(by='feature importance', ascending=False).iloc[:40]

,keyword,feature importance
0,obese,0.199750
1,obesity,0.131942
41,chronic,0.038239
16,mild,0.032329
37,continued,0.030851
7,insulin,0.030134
43,i,0.028346
51,b,0.027809
48,artery,0.027455
11,secondary,0.026585
